In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from collections import Counter

In [3]:
#Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [95]:
import os

In [4]:
#Read the loans data
loans = pd.read_csv('lending-club-data/lending-club-data.csv')

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [6]:
loans.shape

(122607, 68)

In [7]:
loans.iloc[-1:, :]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
122606,9604874,11457002,2000,2000,2000,36 months,7.9,62.59,A,A4,...,0.8,0.0,1.0,1.0,0,0.904916,20170101T000000,0,1,1


## Missing Value Analysis

In [8]:
def find_missing(data):
    missing_count = data.isna().sum()
    missing_percentage = data.isna().sum()*100/data.shape[0]
    missing = pd.DataFrame({'Features': data.columns,
                            'Missing Count': missing_count,
                            'Missing Percentage': missing_percentage},
                             columns = ['Features', 'Missing Count',
                                     'Missing Percentage'])
    missing = missing.sort_values(by='Missing Count',
                                 ascending = False)
    missing = missing.set_index('Features')
    return missing

## Create a New Target - 'safe_loans'

In [9]:
#safe if not bad
#bad if not safe
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: 1 if x==0 else -1)
loans.iloc[:5, -2:]

,last_major_derog_none,safe_loans
0,1,1
1,1,-1
2,1,1
3,1,1
4,1,1


In [10]:
loans.loc[loans.bad_loans==1, :].shape

(23150, 69)

In [11]:
#The following should return an empty data frame
loans.loc[((loans.bad_loans==1) & (loans.safe_loans==1)) | \
         ((loans.bad_loans==0) & (loans.safe_loans==0))].head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans


In [12]:
if('bad_loans' in loans.columns):
    loans = loans.drop('bad_loans', axis=1)
    

In [13]:
'bad_loans' in loans.columns

False

In [14]:
#Percentage of risky and safe loans
risky = loans.loc[loans.safe_loans == -1, 'safe_loans']
safe = loans.loc[loans.safe_loans == 1, 'safe_loans']
print("Bad Loans percent:- %.2f\nSafe Loans percent:- %.2f"\
      %(risky.shape[0]*100/loans.shape[0], safe.shape[0]*100/loans.shape[0]))

Bad Loans percent:- 18.88
Safe Loans percent:- 81.12


In [15]:
risky.shape[0] + safe.shape[0] == loans.shape[0]

True

In [16]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

In [17]:
# Extract the feature columns and target column
loans = loans[features + [target]]
#loans_new = loans_new[features + [target]]

In [18]:
np.unique(loans[target])

array([-1,  1])

In [19]:
features + [target]

['grade',
 'sub_grade',
 'short_emp',
 'emp_length_num',
 'home_ownership',
 'dti',
 'purpose',
 'term',
 'last_delinq_none',
 'last_major_derog_none',
 'revol_util',
 'total_rec_late_fee',
 'safe_loans']

In [20]:
find_missing(loans[features]).head()

,Missing Count,Missing Percentage
Features,,
grade,0,0.0
sub_grade,0,0.0
short_emp,0,0.0
emp_length_num,0,0.0
home_ownership,0,0.0


In [21]:
Counter(loans.safe_loans)

Counter({1: 99457, -1: 23150})

## Apply One Hot Encoding

In [22]:
from sklearn.preprocessing import OneHotEncoder as OHE


In [23]:
loans.dtypes.grade == 'int64'

False

In [24]:
#Find the categorical variables in loans data
categorical = [col for col in features \
               if(loans.dtypes[col] == 'object')]
categorical

['grade', 'sub_grade', 'home_ownership', 'purpose', 'term']

In [25]:
loans[categorical].head()

,grade,sub_grade,home_ownership,purpose,term
0,B,B2,RENT,credit_card,36 months
1,C,C4,RENT,car,60 months
2,C,C5,RENT,small_business,36 months
3,C,C1,RENT,other,36 months
4,A,A4,RENT,wedding,36 months


## One Hot Encoding

In [26]:
#print(pd.get_dummies(loans[categorical_features], prefix=None))
loans_temp = pd.DataFrame(pd.get_dummies(loans[categorical],\
                                         prefix=None))

#Append the dummy columns to the loans data
loans = pd.concat([loans, loans_temp], axis=1)

#Drop the categorical columns from the loans data
loans = loans.drop(categorical, axis=1)

In [27]:
#One Hot Encoded Columns
OHE_cols = list(loans_temp.columns)

In [28]:
OHE_cols

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'sub_grade_A1',
 'sub_grade_A2',
 'sub_grade_A3',
 'sub_grade_A4',
 'sub_grade_A5',
 'sub_grade_B1',
 'sub_grade_B2',
 'sub_grade_B3',
 'sub_grade_B4',
 'sub_grade_B5',
 'sub_grade_C1',
 'sub_grade_C2',
 'sub_grade_C3',
 'sub_grade_C4',
 'sub_grade_C5',
 'sub_grade_D1',
 'sub_grade_D2',
 'sub_grade_D3',
 'sub_grade_D4',
 'sub_grade_D5',
 'sub_grade_E1',
 'sub_grade_E2',
 'sub_grade_E3',
 'sub_grade_E4',
 'sub_grade_E5',
 'sub_grade_F1',
 'sub_grade_F2',
 'sub_grade_F3',
 'sub_grade_F4',
 'sub_grade_F5',
 'sub_grade_G1',
 'sub_grade_G2',
 'sub_grade_G3',
 'sub_grade_G4',
 'sub_grade_G5',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_small_bus

In [90]:
OHE_cols

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'sub_grade_A1',
 'sub_grade_A2',
 'sub_grade_A3',
 'sub_grade_A4',
 'sub_grade_A5',
 'sub_grade_B1',
 'sub_grade_B2',
 'sub_grade_B3',
 'sub_grade_B4',
 'sub_grade_B5',
 'sub_grade_C1',
 'sub_grade_C2',
 'sub_grade_C3',
 'sub_grade_C4',
 'sub_grade_C5',
 'sub_grade_D1',
 'sub_grade_D2',
 'sub_grade_D3',
 'sub_grade_D4',
 'sub_grade_D5',
 'sub_grade_E1',
 'sub_grade_E2',
 'sub_grade_E3',
 'sub_grade_E4',
 'sub_grade_E5',
 'sub_grade_F1',
 'sub_grade_F2',
 'sub_grade_F3',
 'sub_grade_F4',
 'sub_grade_F5',
 'sub_grade_G1',
 'sub_grade_G2',
 'sub_grade_G3',
 'sub_grade_G4',
 'sub_grade_G5',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_small_bus

In [91]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
assignment_2_cols = [col for col in OHE_cols \
                    if(col.startswith('grade') or\
                       col.startswith('term') or\
                       col.startswith('home_ownership') or\
                       col.startswith('emp_length'))]

In [92]:
assignment_2_cols

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'term_ 36 months',
 'term_ 60 months']

In [29]:
'safe_loans' in OHE_cols

False

In [30]:
set(loans.columns).intersection(set(categorical))

set()

In [31]:
numeric_features = list(set(features).difference(set(categorical)))
numeric_features 

['last_major_derog_none',
 'emp_length_num',
 'revol_util',
 'total_rec_late_fee',
 'short_emp',
 'dti',
 'last_delinq_none']

In [32]:
loans[numeric_features + OHE_cols]

,last_major_derog_none,emp_length_num,revol_util,total_rec_late_fee,short_emp,dti,last_delinq_none,grade_A,grade_B,grade_C,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,1,11,83.70,0.00,0,27.65,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,9.40,0.00,1,1.00,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,1,11,98.50,0.00,0,8.72,1,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,1,11,21.00,16.97,0,20.00,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
4,1,4,28.30,0.00,0,11.20,1,1,0,0,...,0,0,0,0,0,0,0,1,1,0
5,1,10,87.50,0.00,0,5.35,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,1,5,32.60,0.00,0,5.55,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
7,1,1,36.50,0.00,1,18.08,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
8,1,6,20.60,0.00,0,16.12,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
9,1,11,67.10,0.00,0,10.78,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [33]:

set(loans.columns).intersection(set(numeric_features + OHE_cols)) == \
set(numeric_features + OHE_cols)

True

In [34]:
'safe_loans' in [numeric_features + OHE_cols]

False

In [35]:
train_index = pd.read_json('assignment-1-train-idx.json/assignment-1-train-idx.json')
valid_index = pd.read_json('assignment-1-validation-idx.json/assignment-1-validation-idx.json')


In [36]:
valid_index

,0
0,24
1,41
2,60
3,93
4,132
5,160
6,162
7,175
8,184
9,270


In [37]:
train_index

,0
0,1
1,6
2,7
3,10
4,12
5,18
6,21
7,23
8,45
9,48


In [38]:
#loans = loans[numeric_features + OHE_cols + '']

In [39]:
target

'safe_loans'

In [40]:
loans[numeric_features + OHE_cols + [target]]

,last_major_derog_none,emp_length_num,revol_util,total_rec_late_fee,short_emp,dti,last_delinq_none,grade_A,grade_B,grade_C,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months,safe_loans
0,1,11,83.70,0.00,0,27.65,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1
1,1,1,9.40,0.00,1,1.00,1,0,0,1,...,0,0,0,0,0,0,0,0,1,-1
2,1,11,98.50,0.00,0,8.72,1,0,0,1,...,0,0,0,0,1,0,0,1,0,1
3,1,11,21.00,16.97,0,20.00,0,0,0,1,...,0,0,0,1,0,0,0,1,0,1
4,1,4,28.30,0.00,0,11.20,1,1,0,0,...,0,0,0,0,0,0,1,1,0,1
5,1,10,87.50,0.00,0,5.35,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
6,1,5,32.60,0.00,0,5.55,1,0,0,0,...,0,0,0,0,1,0,0,0,1,-1
7,1,1,36.50,0.00,1,18.08,1,0,1,0,...,0,0,0,1,0,0,0,0,1,-1
8,1,6,20.60,0.00,0,16.12,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1
9,1,11,67.10,0.00,0,10.78,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [41]:
train_data = loans.loc[train_index[0], numeric_features + OHE_cols + [target]]
validation_data = loans.loc[valid_index[0], numeric_features + OHE_cols + [target]]

In [96]:
train_data[assignment_2_cols + [target]].to_csv(os.getcwd() + '/train.csv')

In [97]:
validation_data[assignment_2_cols + [target]].to_csv(os.getcwd() + '/validation.csv')

In [98]:
train_data.shape

(37224, 68)

In [100]:
validation_data.shape

(9284, 68)

## Decision Tree Classifier

In [42]:
from sklearn.tree import DecisionTreeClassifier as DTC

In [43]:
help(DTC)

Help on class DecisionTreeClassifier in module sklearn.tree.tree:

class DecisionTreeClassifier(BaseDecisionTree, sklearn.base.ClassifierMixin)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : string, optional (default="best")
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_depth : int or None, optional (default=None)
 |      The maximum depth of the tree. If None, then nodes are expanded until
 |      all leaves are pure or until all leaves contain less than
 |      min_samples_split samples.
 |  
 |  min_samples_split : int, float, optional (default=2)

In [44]:
class Modelling():
    def __init__(self, X_train, y_train, X_val, y_val, model):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.model = model
    
    def evaluate(self, predicted):
        CM = confusion_matrix(self.y_val, predicted)
        print("Confusion Matrix:-\n")
        
        print(CM)
        return (accuracy_score(self.y_val, predicted))
        
    def prediction(self):
        self.model.fit(self.X_train, self.y_train)
        
        prediction = self.model.predict(self.X_val)
        
        print("Accuracy Score:-%.2f\n" %(self.evaluate(prediction)))

In [45]:
X_train, y_train = train_data.iloc[:, :-1], train_data.iloc[:, -1]
X_val, y_val = validation_data.iloc[:, :-1], validation_data.iloc[:, -1]
model = Modelling(X_train, y_train, X_val, y_val, decision_tree_model)


NameError: name 'decision_tree_model' is not defined

In [87]:
def evaluate(actual, predicted):
    CM = confusion_matrix(actual, predicted)
    
    print("Confusion Matrix:-\n")

    print(CM)
    tn, fp, fn, tp = CM.ravel() 
    print("TN:-%d\nFP:-%d\nFN:-%d\nTP:-%d\n" %(tn, fp, fn, tp))
    
    print("Accuracy:-\n")
    return (accuracy_score(actual, predicted))

In [47]:
def prediction(model, X, y):
    model.fit(X, y)
    predicted = model.predict(X)
    
    evaluate(model, y, predicted)

### decision_tree_model - Model with depth=6

In [48]:
decision_tree_model = DTC(max_depth=6)


In [49]:
y_train

1        -1
6        -1
7        -1
10       -1
12       -1
18       -1
21       -1
23       -1
45       -1
48       -1
50       -1
58       -1
63       -1
87       -1
89       -1
102      -1
108      -1
111      -1
118      -1
124      -1
136      -1
138      -1
140      -1
151      -1
158      -1
159      -1
165      -1
166      -1
169      -1
183      -1
         ..
122433    1
122436    1
122454    1
122458    1
122465    1
122473    1
122484    1
122489    1
122494    1
122498    1
122500    1
122501    1
122505    1
122506    1
122513    1
122520    1
122525    1
122531    1
122538    1
122544    1
122552    1
122553    1
122558    1
122561    1
122569    1
122572    1
122575    1
122588    1
122599    1
122603    1
Name: safe_loans, Length: 37224, dtype: int64

In [51]:
decision_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## First, let's grab 2 positive examples and 2 negative examples from the Validation Set.

In [52]:
validation_safe_loans = validation_data[validation_data['safe_loans']==1]
validation_risky_loans = validation_data[validation_data['safe_loans']==-1]

sample_validation_data_risky = validation_risky_loans.iloc[:2, :]
sample_validation_data_safe = validation_safe_loans.iloc[:2, :]

sample_validation_data = pd.concat([sample_validation_data_safe,\
                                    sample_validation_data_risky], axis = 0)

In [53]:
sample_validation_data.head()

,last_major_derog_none,emp_length_num,revol_util,total_rec_late_fee,short_emp,dti,last_delinq_none,grade_A,grade_B,grade_C,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months,safe_loans
19,1,11,82.4,0.0,0,11.18,1,0,1,0,...,0,0,0,0,0,0,0,1,0,1
79,1,10,96.4,0.0,0,16.85,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
24,1,3,59.5,0.0,0,13.97,0,0,0,0,...,0,0,0,1,0,0,0,0,1,-1
41,1,11,62.1,0.0,0,16.33,1,1,0,0,...,0,0,0,0,0,0,0,1,0,-1


## Prediction using the trained model on 'sample validation data'

In [54]:
def validation_results(model):
    sample_validation_prediction = model.predict(
                                             sample_validation_data.iloc\
                                             [:, :-1])
    
    probability = model.predict_proba(sample_validation_data.iloc[:, :-1])
    
    unsafe_probability = [prob[0] for prob in probability]
    safe_probability = [prob[1] for prob in probability]

    sample_validation_result = pd.DataFrame({'Actual':\
                                             sample_validation_data['safe_loans'],
                                         'Predicted':\
                                             sample_validation_prediction,
                                         'Probability Unsafe':\
                                             unsafe_probability,
                                         'Probability Safe':\
                                             safe_probability},
                                        columns = ['Actual', 'Predicted',
                                                   'Probability Unsafe', 
                                                   'Probability Safe'])
    return (sample_validation_result)

### decision_tree_model

In [55]:
decision_tree_model = DTC(max_depth=6)

In [56]:
decision_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [57]:
#Results on sampled validation data
validation_results(decision_tree_model)

,Actual,Predicted,Probability Unsafe,Probability Safe
19,1,1,0.341565,0.658435
79,1,-1,0.536306,0.463694
24,-1,-1,0.647510,0.352490
41,-1,1,0.207895,0.792105


### Accuracy - Training Data

In [88]:
predicted_train_decision_tree = decision_tree_model.predict(X_train)
print(evaluate(predicted_train_decision_tree, y_train))

Confusion Matrix:-

[[11708  6613]
 [ 6768 12135]]
TN:-11708
FP:-6613
FN:-6768
TP:-12135

Accuracy:-

0.6405276165914464


In [73]:
np.round(0.6405276165914464, 2)

0.64

### Accuracy - Validation Data

In [89]:
predicted_val_decision_tree = decision_tree_model.predict(X_val)
np.round(evaluate(predicted_val_decision_tree, y_val), 2)

Confusion Matrix:-

[[3013 1717]
 [1661 2893]]
TN:-3013
FP:-1717
FN:-1661
TP:-2893

Accuracy:-



0.64

In [83]:
help(confusion_matrix)

Help on function confusion_matrix in module sklearn.metrics.classification:

confusion_matrix(y_true, y_pred, labels=None, sample_weight=None)
    Compute confusion matrix to evaluate the accuracy of a classification
    
    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` but
    predicted to be in group :math:`j`.
    
    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.
    
    Read more in the :ref:`User Guide <confusion_matrix>`.
    
    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (correct) target values.
    
    y_pred : array, shape = [n_samples]
        Estimated targets as returned by a classifier.
    
    labels : array, shape = [n_classes], optional
        List of labels to index the m

## Quiz Question:- What is the total cost of mistakes made by decision_tree_model on validation_data? Please enter your answer as a plain integer, without the dollar sign or the comma separator, e.g. 3002000.

Assume a cost of $10,000 per false negative.

Assume a cost of $20,000 per false positive.

In [84]:
fn, fn_cost = 1661, 10000
fp, fp_cost = 1717, 20000
print("Total Cost = %d" %((fn * fn_cost) + (fp * fp_cost)))

Total Cost = 50950000


### small model Model with depth=2

In [60]:
small_model = DTC(max_depth=2)

In [61]:
small_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [62]:
#Results on sampled validation data
validation_results(small_model)

,Actual,Predicted,Probability Unsafe,Probability Safe
19,1,1,0.418966,0.581034
79,1,-1,0.592553,0.407447
24,-1,-1,0.592553,0.407447
41,-1,1,0.231201,0.768799


### Accuracy - Training Data

In [79]:
predicted_train_small = small_model.predict(X_train)
np.round(evaluate(predicted_train_small, y_train), 2)

Confusion Matrix:-

[[12980  8891]
 [ 5496  9857]]
Accuracy:-



0.61

### Accuracy - Validation Data

In [80]:
predicted_val_small = small_model.predict(X_val)
np.round(evaluate(predicted_val_small, y_val), 2)

Confusion Matrix:-

[[3342 2202]
 [1332 2408]]
Accuracy:-



0.62

## Evaluating accuracy of a complex decision tree model
Big Model with max_depth=10

In [65]:
big_model = DTC(max_depth=10)

In [66]:
big_model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [67]:
#Results on sampled validation data
validation_results(big_model)

,Actual,Predicted,Probability Unsafe,Probability Safe
19,1,1,0.367673,0.632327
79,1,-1,0.710526,0.289474
24,-1,-1,0.644295,0.355705
41,-1,1,0.219677,0.780323


### Accuracy - Training Data

In [81]:
predicted_train_big = big_model.predict(X_train)
np.round(evaluate(predicted_train_big, y_train), 2)

Confusion Matrix:-

[[12482  6523]
 [ 5994 12225]]
Accuracy:-



0.66

### Accuracy - Validation Data

In [82]:
predicted_val_big = big_model.predict(X_val)
np.round(evaluate(predicted_val_big, y_val), 2)

Confusion Matrix:-

[[3031 1818]
 [1643 2792]]
Accuracy:-



0.63

### Quiz Question: How does the performance of big_model on the validation set compare to decision_tree_model on the validation set? Is this a sign of overfitting?

In [71]:
print("big_model Accuracy is %.2f\ndecision_tree_model accuracy is %.2f"\
     %(62.72, 63.61))

big_model Accuracy is 62.72
decision_tree_model accuracy is 63.61
